In [11]:
from ortools.linear_solver import pywraplp
import os
import pandas as pd
import numpy as np
import slim_os 
import slim_python as slim

In [15]:
cd StageDaniel

C:\Users\danie\Documents\StageDaniel


In [16]:
def load_data(name='breastcancer'):
# requirements for CSV data file
# - outcome variable in first column
# - outcome variable values should be [-1, 1] or [0, 1]
# - first row contains names for the outcome variable + input variables
# - no empty cells
    data_name = name
    data_dir = os.getcwd() + '/data/'
    data_csv_file = data_dir + data_name + '_processed.csv'

    # load data file from csv
    df = pd.read_csv(data_csv_file, sep = ',')
    data = df.as_matrix()
    data_headers = list(df.columns.values)
    N = data.shape[0]

    # setup Y vector and Y_name
    Y_col_idx = [0]
    Y = data[:, Y_col_idx]
    Y_name = [data_headers[j] for j in Y_col_idx]
    Y[Y == 0] = -1

    # setup X and X_names
    X_col_idx = [j for j in range(data.shape[1]) if j not in Y_col_idx]
    X = data[:, X_col_idx]
    X_names = [data_headers[j] for j in X_col_idx]

    # insert a column of ones to X for the intercept
    X = np.insert(arr = X, obj = 0, values = np.ones(N), axis = 1)
    X_names.insert(0, '(Intercept)')

    # run sanity checks
    slim.check_data(X = X, Y = Y, X_names = X_names)
    
    return (X, X_names, Y, Y_name)



In [17]:
(X, X_names, Y, Y_name) = load_data(name='haberman')

C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


Get coefficient constraints and set slim parameters

In [19]:

from slim_python.helper_functions import *
from slim_python.SLIMCoefficientConstraints import SLIMCoefficientConstraints
from math import ceil, floor
print_flag = False

coef_constraints = slim.SLIMCoefficientConstraints(variable_names = X_names, ub = 5, lb = -5)
#choose upper and lower bounds for the intercept coefficient
#to ensure that there will be no regularization due to the intercept, choose
#
#intercept_ub < min_i(min_score_i)
#intercept_lb > max_i(max_score_i)
#
#where min_score_i = min((Y*X) * \rho) for rho in \Lset
#where max_score_i = max((Y*X) * \rho) for rho in \Lset
#
#setting intercept_ub and intercept_lb in this way ensures that we can always
# classify every point as positive and negative
scores_at_ub = (Y * X) * coef_constraints.ub
scores_at_lb = (Y * X) * coef_constraints.lb
non_intercept_ind = np.array([n != '(Intercept)' for n in X_names])
scores_at_ub = scores_at_ub[:, non_intercept_ind]
scores_at_lb = scores_at_lb[:, non_intercept_ind]
max_scores = np.fmax(scores_at_ub, scores_at_lb)
min_scores = np.fmin(scores_at_ub, scores_at_lb)
max_scores = np.sum(max_scores, 1)
min_scores = np.sum(min_scores, 1)

intercept_ub = -min(min_scores) + 1
intercept_lb = -max(max_scores) + 1
coef_constraints.set_field('ub', '(Intercept)', intercept_ub)
coef_constraints.set_field('lb', '(Intercept)', intercept_lb)

slim_input = {
    'X': X,
    'X_names': X_names,
    'Y': Y,
    'C_0': 0.001,
    'w_pos': 1.0,
    'w_neg': 1.0,
    'L0_min': 0,
    'L0_max': float('inf'),
    'err_min': 0,
    'err_max': 1.0,
    'pos_err_min': 0,
    'pos_err_max': 1.0,
    'neg_err_min': 0,
    'neg_err_max': 1.0,
    'coef_constraints': coef_constraints
}


Create SLIM

In [20]:
# Create SLIM

from slim_python.helper_functions import *
from slim_python.SLIMCoefficientConstraints import SLIMCoefficientConstraints
from math import ceil, floor
input = slim_input
print_flag = False


"""
:param input: dictionary with the following keys
%Y          N x 1 np.array of labels (-1 or 1 only)
%X          N x P np.matrix of feature values (should include a column of 1s to act as an intercept
%X_names    P x 1 list of strings with names of the feature values (all unique and Intercept name)

:return:
%slim_IP
%slim_info
"""

#check preconditions
assert 'X' in input, 'no field named X  in input'
assert 'X_names' in input, 'no field named X_names in input'
assert 'Y' in input, 'no field named Y in input'
assert input['X'].shape[0] == input['Y'].shape[0]
assert input['X'].shape[1] == len(input['X_names'])
assert all((input['Y'] == 1) | (input['Y'] == -1))

XY = input['X'] * input['Y']

#sizes
N = input['X'].shape[0]
P = input['X'].shape[1]
pos_ind = np.flatnonzero(input['Y'] == 1)
neg_ind = np.flatnonzero(input['Y'] == -1)
N_pos = len(pos_ind)
N_neg = len(neg_ind)
binary_data_flag = np.all((input['X'] == 0) | (input['X'] == 1))

#outcome variable name
if ('Y_name' in input) and (type(input['Y_name']) is list):
    input['Y_name'] = input['Y_name'][0]
elif ('Y_name' in input) and (type(input['Y_name']) is str):
    pass
else:
    input['Y_name'] = 'Outcome'

#TODO: check intercept conditions
## first column of X should be all 1s
## first element of X_name should be '(Intercept)'

#set default parameters
input = get_or_set_default(input, 'C_0', 0.01, print_flag = print_flag)
input = get_or_set_default(input, 'w_pos', 1.0, print_flag = print_flag)
input = get_or_set_default(input, 'w_neg', 2.0 - input['w_pos'], print_flag = print_flag)
input = get_or_set_default(input, 'L0_min', 0, print_flag = print_flag)
input = get_or_set_default(input, 'L0_max', P, print_flag = print_flag)
input = get_or_set_default(input, 'err_min', 0.00, print_flag = print_flag)
input = get_or_set_default(input, 'err_max', 1.00, print_flag = print_flag)
input = get_or_set_default(input, 'pos_err_min', 0.00, print_flag = print_flag)
input = get_or_set_default(input, 'pos_err_max', 1.00, print_flag = print_flag)
input = get_or_set_default(input, 'neg_err_min', 0.00, print_flag = print_flag)
input = get_or_set_default(input, 'neg_err_max', 1.00, print_flag = print_flag)

#internal parameters
input = get_or_set_default(input, 'C_1', float('nan'), print_flag = print_flag)
input = get_or_set_default(input, 'M', float('nan'), print_flag = print_flag)
input = get_or_set_default(input, 'epsilon', 0.001, print_flag = print_flag)

#coefficient constraints
if 'coef_constraints' in input:
    coef_constraints = input['coef_constraints']
else:
    coef_constraints = SLIMCoefficientConstraints(variable_names = input['X_names'])


assert len(coef_constraints) == P

# bounds
rho_lb = np.array(coef_constraints.lb)
rho_ub = np.array(coef_constraints.ub)
rho_max = np.maximum(np.abs(rho_lb), np.abs(rho_ub))
beta_ub = rho_max
beta_lb = np.zeros_like(rho_max)
beta_lb[rho_lb > 0] = rho_lb[rho_lb > 0]
beta_lb[rho_ub < 0] = rho_ub[rho_ub < 0]

# signs
signs = coef_constraints.sign
sign_pos = signs == 1
sign_neg = signs == -1

#types
types = coef_constraints.get_field_as_list('vtype')
rho_type = ''.join(types)
#TODO: add support for custom variable types

#class-based weights
w_pos = input['w_pos']
w_neg = input['w_neg']
w_total = w_pos + w_neg
w_pos = 2.0 * (w_pos/w_total)
w_neg = 2.0 * (w_neg/w_total)
assert w_pos > 0.0
assert w_neg > 0.0
assert w_pos + w_neg == 2.0

#L0 regularization penalty
C_0j = np.copy(coef_constraints.C_0j)
L0_reg_ind = np.isnan(C_0j)
C_0j[L0_reg_ind] = input['C_0']
C_0 = C_0j
assert(all(C_0[L0_reg_ind] > 0.0))

#L1 regularization penalty
L1_reg_ind = L0_reg_ind
if not np.isnan(input['C_1']):
    C_1 = input['C_1']
else:
    C_1 = 0.5 * min(w_pos/N, w_neg/N, min(C_0[L1_reg_ind] / np.sum(rho_max)))
C_1 = C_1 * np.ones(shape = (P,))
C_1[~L1_reg_ind] = 0.0
assert(all(C_1[L1_reg_ind] > 0.0))

# model size bounds
L0_min = max(input['L0_min'], 0.0)
L0_max = min(input['L0_max'], np.sum(L0_reg_ind))
L0_min = ceil(L0_min)
L0_max = floor(L0_max)
assert(L0_min <= L0_max)

# total positive error bounds
pos_err_min = 0.0 if np.isnan(input['pos_err_min']) else input['pos_err_min']
pos_err_max = 1.0 if np.isnan(input['pos_err_max']) else input['pos_err_max']
pos_err_min = max(ceil(N_pos*pos_err_min), 0)
pos_err_max = min(floor(N_pos*pos_err_max), N_pos)

# total negative error bounds
neg_err_min = 0.0 if np.isnan(input['neg_err_min']) else input['neg_err_min']
neg_err_max = 1.0 if np.isnan(input['neg_err_max']) else input['neg_err_max']
neg_err_min = max(ceil(N_neg*neg_err_min), 0)
neg_err_max = min(floor(N_neg*neg_err_max), N_neg)

# total error bounds
err_min = 0.0 if np.isnan(input['err_min']) else input['err_min']
err_max = 1.0 if np.isnan(input['err_max']) else input['err_max']
err_min = max(ceil(N*err_min), 0)
err_max = min(floor(N*err_max), N)

# sanity checks for error bounds
assert(err_min <= err_max)
assert(pos_err_min <= pos_err_max)
assert(neg_err_min <= neg_err_max)
assert(err_min >= 0)
assert(pos_err_min >= 0)
assert(neg_err_min >= 0)
assert(err_max <= N)
assert(pos_err_max <= N_pos)
assert(neg_err_max <= N_neg)

#TODO: strengthen bounds
#loss constraint parameters
epsilon  = input['epsilon']
if np.isnan(input['M']):
    max_points = np.maximum(XY * rho_lb, XY * rho_ub)
    max_score_reg = np.sum(-np.sort(-max_points[:, L0_reg_ind])[:, 0:int(L0_max)], axis = 1)
    max_score_no_reg = np.sum(max_points[:, ~L0_reg_ind], axis = 1)
    max_score = max_score_reg + max_score_no_reg
    M = max_score + 1.05 * epsilon
else:
    M = input['M']

#sanity checks for loss constraint parameters
M = M * np.ones(shape = (N,))
M_max = max(np.sum(abs(XY) * rho_max, axis = 1)) + 1.05 * input['epsilon']
assert(len(M) == N)
assert(all(M > 0))
assert(all(M <= M_max))
assert(epsilon > 0.0)
assert(epsilon < 1.0)



In [21]:
#objective costs (we solve min total_error + N * C_0 * L0_norm + N
err_cost = np.ones(shape = (N,))
err_cost[pos_ind] = w_pos
err_cost[neg_ind] = w_neg
C_0 = N * C_0
C_1 = N * C_1

# #variable-related values
# obj = [0.0] * P + C_0.tolist() + C_1.tolist() + err_cost.tolist()
# ub = rho_ub.tolist() + [1] * P + beta_ub.tolist() + [1] * N
# lb = rho_lb.tolist() + [0] * P + beta_lb.tolist() + [0] * N
# ctype  = rho_type + 'B'*P + 'C'*P + 'B'*N

#variable-related names
rho_names   = ['rho_' + str(j) for j in range(0, P)]
alpha_names = ['alpha_' + str(j) for j in range(0, P)]
beta_names = ['beta_' + str(j) for j in range(0, P)]
error_names = ['error_' + str(i) for i in range(0, N)]
var_names = rho_names + alpha_names + beta_names + error_names

#variable-related error checking
n_var = 3 * P + N
# assert(len(obj) == n_var)
# assert(len(ub) == n_var)
# assert(len(lb) == n_var)
# assert(len(ctype) == n_var)
assert(len(var_names) == n_var)

In [22]:
XY

array([[  1,  30,   1,   6],
       [  1,  30,   3,   4],
       [  1,  30,   0,   7],
       ...,
       [  1,  77,   3,   7],
       [ -1, -78,  -1,  -7],
       [ -1, -83,  -2,   0]], dtype=int64)

In [23]:
slim_IP = pywraplp.Solver('slim_ip_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [24]:
#add variables
slim_IP.Clear()
rhos = []
alphas = []
betas = []
errors = []
for j in range(P):
    rhos.append(slim_IP.IntVar(rho_lb[j], rho_ub[j], rho_names[j]))
for j in range(P):
    alphas.append(slim_IP.BoolVar( alpha_names[j]))
for j in range(P):
    betas.append(slim_IP.IntVar(beta_lb[j], beta_ub[j], beta_names[j]))
for i in range(N):
    errors.append(slim_IP.BoolVar(error_names[i]))


In [25]:
#Loss Constraints
#Enforce z_i = 1 if incorrect classification)
#M_i * z_i >= XY[i,].dot(rho) + epsilon
for i in range(N):
    loss_constraint = slim_IP.Constraint(epsilon, slim_IP.infinity(), "error_" + str(i))
    loss_constraint.SetCoefficient(errors[i], M[i])
    for j in range(P):
        loss_constraint.SetCoefficient(rhos[j], XY[i][j]/1)
        
# 0-Norm LB Constraints:
# lambda_j,lb * alpha_j <= lambda_j <= Inf
# 0 <= lambda_j - lambda_j,lb * alpha_j < Inf
for j in range(P):
    zero_norm_lb_constraint = slim_IP.Constraint(0, slim_IP.infinity(), "L0_norm_lb_" + str(j))
    zero_norm_lb_constraint.SetCoefficient(rhos[j], 1)
    zero_norm_lb_constraint.SetCoefficient(alphas[j], -rho_lb[j])

# 0-Norm UB Constraints:
# lambda_j <= lambda_j,ub * alpha_j
# 0 <= -lambda_j + lambda_j,ub * alpha_j
for j in range(P):
    zero_norm_ub_constraint = slim_IP.Constraint(0, slim_IP.infinity(), "L0_norm_ub_" + str(j))
    zero_norm_ub_constraint.SetCoefficient(rhos[j], -1)
    zero_norm_ub_constraint.SetCoefficient(alphas[j], rho_ub[j])
    
# 1-Norm Positive Constraints:
#actual constraint: lambda_j <= beta_j
#cplex constraint:  0 <= -lambda_j + beta_j <= Inf
for j in range(P):
    one_norm_pos_constraint = slim_IP.Constraint(0, slim_IP.infinity(), "L1_norm_pos_" + str(j))
    one_norm_pos_constraint.SetCoefficient(rhos[j], -1 )
    one_norm_pos_constraint.SetCoefficient(betas[j], 1 )
    
# 1-Norm Negative Constraints:
#actual constraint: -lambda_j <= beta_j
#cplex constraint:  0 <= lambda_j + beta_j <= Inf
for j in range(P):
    one_norm_neg_constraint = slim_IP.Constraint(0, slim_IP.infinity(), "L1_norm_neg_" + str(j))
    one_norm_neg_constraint.SetCoefficient(rhos[j], 1 )
    one_norm_neg_constraint.SetCoefficient(betas[j], 1 )
    


In [26]:
objective = slim_IP.Objective() 
for j in range(P):
    objective.SetCoefficient(alphas[j], C_0[j])
for j in range(P):
    objective.SetCoefficient(betas[j], C_1[j])
for i in range(N):
    objective.SetCoefficient(errors[i], err_cost[i])
objective.SetOptimizationDirection(False)


In [27]:
seconds = 20
slim_IP.SetTimeLimit(seconds*1000)

In [28]:
print("Start time: ", time.ctime())
slim_IP.Solve()
print("Finished at time: ", time.ctime())

Start time:  Thu Jul  9 13:37:00 2020
Finished at time:  Thu Jul  9 13:37:20 2020


In [29]:
objective.Value()

75.61309090909091

In [30]:
rho_solution2 = np.array([rhos[j].SolutionValue() for j in range(P)])
print(rho_solution2)


total_error = 0
for idx, error in enumerate(errors):
    total_error += error.solution_value()
#     print('Scoring sum: ', X[idx].dot(rho_solution))
#     print('Error: ',error.solution_value())
#     print('y: ', Y[idx])
print((N-total_error)/N)

[32.  0. -3. -1.]
0.7549019607843137


In [55]:
# help(slim_IP)

In [56]:
# for i in range(N):
#     print(slim_IP.constraints()[i].name())
#     print(slim_IP.constraints()[i].GetCoefficient(errors[i]))

In [57]:
obs = 0
M[obs]*errors[obs].solution_value()

0.0

In [58]:
errors[0].solution_value()
err_cost[0]

1.0

In [64]:
(N-Total_error)/N

0.7581699346405228

In [62]:
Total_error = 0
for idx, error in enumerate(errors):
#     print('Scoring sum: ', X[idx].dot(rho_solution))
    Total_error += error.solution_value()
#     print('Error: ',error.solution_value())
#     print('y: ', Y[idx])


74.0

In [370]:
for variable in alphas:
    print(variable.name(), variable.solution_value())

alpha_0 1.0
alpha_1 0.0
alpha_2 0.0
alpha_3 0.0
alpha_4 0.0
alpha_5 0.0
alpha_6 0.0
alpha_7 0.0
alpha_8 0.0
alpha_9 0.0


In [373]:
err_cost

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [195]:
XY

array([[  1,  30,   1,   6],
       [  1,  30,   3,   4],
       [  1,  30,   0,   7],
       ...,
       [  1,  77,   3,   7],
       [ -1, -78,  -1,  -7],
       [ -1, -83,  -2,   0]], dtype=int64)

In [222]:
M

array([ 731.00105,  731.00105,  731.00105,  716.00105,  756.00105,
        761.00105,  721.00105,  719.00105,  799.00105,  866.00105,
        731.00105,  781.00105,  796.00105,  726.00105,  816.00105,
        746.00105,  741.00105,  781.00105,  741.00105,  756.00105,
        731.00105,  766.00105,  816.00105,  756.00105,  894.00105,
        751.00105,  746.00105,  746.00105,  771.00105,  771.00105,
        776.00105,  831.00105,  751.00105,  806.00105,  779.00105,
        766.00105,  786.00105,  741.00105,  756.00105,  786.00105,
        756.00105,  746.00105,  781.00105,  874.00105,  779.00105,
        794.00105,  751.00105,  796.00105,  756.00105,  781.00105,
        846.00105,  786.00105,  786.00105,  814.00105,  759.00105,
        756.00105,  771.00105,  771.00105,  791.00105,  876.00105,
        791.00105,  786.00105, 1019.00105,  774.00105,  789.00105,
        789.00105,  856.00105,  801.00105,  806.00105,  771.00105,
        796.00105,  796.00105,  821.00105,  824.00105,  809.00